# Курсовая работа. Тетрадь 1

## 1.1 Разбор кода

In [3]:
import re, string, csv, codecs, requests, json
from pymystem3 import Mystem
import pandas as pd
import numpy as np
import time
from tqdm.auto import tqdm

In [4]:
# Кусочек кода, необходимый для решения проблем с сертификатом при загрузке данных
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

Подгружаю фрагмент базы, сожержащий в себе 20 000 строк и удаляю те, которые в поле `Occupation` принимают значение `NULL`

In [152]:
small_base = pd.read_csv('/Users/annalevina/Desktop/thousands.csv', low_memory=False) 

In [8]:
full_base = pd.read_csv('https://www.dropbox.com/s/kpvs0uj3d9a8qar/base_crcase_data.csv?dl=1',
                        low_memory=False, sep = '\t', error_bad_lines=False) 

b'Skipping line 19503: expected 35 fields, saw 50\nSkipping line 779190: expected 35 fields, saw 45\nSkipping line 783770: expected 35 fields, saw 44\nSkipping line 784161: expected 35 fields, saw 59\nSkipping line 788084: expected 35 fields, saw 45\nSkipping line 1625475: expected 35 fields, saw 49\n'


In [7]:
occupations = full_base.Occupation.unique()
occupations = list(occupations[~pd.isnull(occupations)])

`false_subs` и `true_subs` выполняют функции "положительных" и "отрицательных" наборов исключений

In [5]:
m_cache = {}
m = Mystem()
false_subs = set(["с", "в", "к", "г", "д", "е", "ё", "з", "й", "л", "м", "н", "п", "р", "с", "т", "ф", "х", "ц", "ч", "ш", "щ", "ы", "ю", "данна", "ред", "франс", "ст", "главный", "старший",
"ответ", "мл", "младший", "обл", "з-д", "эл"])
true_subs = set(["управляющий", "заключенный", "учащийся", "учащаяся", "рабочий", "рабочая", "заведующий", "заведующая", "дверевой", "нач-к", "пред",
"монашка", "ком"])

При подаче строки спеллер исправляет опечатки, а после заменяет правильными словами (написание которых ему известно) те же места в строке. В выдаче он прописывает места замены строки (начало и конец, посимвольно), а также печатает информацию о том, что было до замены и что стало после нее. В версии ребят в случае, когда после слова шла запятая или иной знак препинания, спеллер производил операцию исправления неверно записанного слова, но, когда он проходился по строке с целью замены, он пропускал его, так как не узнавал вместе со знаками препинания. Другими словами, он искал паттерн “слово”, а встречал “слово,”. На мой взгляд, это связано с тем, что способ, каким спеллер разбивает строки на токены отличается от того, как это делает человек.
В моем варианте, при замене исправленного слова, код опирается на определенные места в строке, которые он предварительно выявил сам, а не разбивает строку на токены повторно. Этот процесс помогает не пропустить при исправлении слова, после которых прописаны знаки препинания и исключает возможность оставить такие слова написанными с ошибками. 
Кроме того, удалось сократить код, так как после изменений не потребовалось учитывать особенности слов, имеющих дефисное написание (например, секретарь-машинистка).

In [10]:
def get_batch_scores(sentence):
	final_sentence = sentence
	i = 5
	while True:
		try:
			r = requests.get("http://speller.yandex.net/services/spellservice.json/checkText?text=" + sentence)
			break
		except:
			print("SLEEP")            
			time.sleep(i) 
			i = i*2
	true_spelling = r.json()
	add = 0
	if true_spelling == []:
		return sentence
	else:
		for dicti in true_spelling:
			if dicti['s'] != []:
				final_sentence = final_sentence[0:(dicti['pos'] + add)] + dicti['s'][0] +\
                final_sentence[(dicti['pos'] + add + len(dicti['word'])):]
				add += len(dicti['s'][0]) - len(dicti['word'])
	return final_sentence

In [11]:
get_batch_scores("чиловек,")

'человек,'

In [8]:
def get_batch_scores(sentence):
	my_dict, final_sentence = {}, []
	r = requests.get("http://speller.yandex.net/services/spellservice.json/checkText?text=" + sentence)
	true_spelling = r.json()
	if true_spelling == []:
		return sentence
	else:
		for dicti in true_spelling:
			if dicti['s'] != []:
				preliminary_dict = {}
				preliminary_dict[dicti['word']] = dicti['s'][0]
				my_dict.update(preliminary_dict)
	for word in sentence.split(" "):
		if word in my_dict:
			final_sentence.append(my_dict[word])
		elif word.split("-")[0] in my_dict:
			final_sentence.append(my_dict[word.split("-")[0]] + "-" + word.split("-")[1])
		else:
			final_sentence.append(word)
	return " ".join(final_sentence)

In [9]:
get_batch_scores("чиловек,")

'чиловек,'

Следующую функцию оставляем без исправлений, она рабочая, ее задача - базовая предобработка текста (удаление некоторой пунктуации, понижение регистра, удаление пробелов и замена пробелоподобных символов и прочее. 

In [13]:
def basic_preprocessing(line):
	remove = string.punctuation
	line = line.lower()
	remove = remove.replace(",", "").replace("-", "").replace(":", "")
	pattern = "[{}]".format(remove)
	line = re.sub(pattern, ' ', line)
	line = re.sub("\s+", " ", line)
	line = line.strip()
	line = line.replace("ё", "е")
	return line

Далее - две функции лемматизации слов. Первый из них - для лемматизации слов с дефисным написанием, второй - общий лемматизатор, включающий в себя первый.

In [14]:
def hyphen_lemmatizer(token):
	try:
		lemma = m.lemmatize(token)[0]
		tokens = token.split("-")
		if len(lemma.split("-")) != len(tokens):
			for i, word in enumerate(tokens):
				try:
					word_lemma = m.lemmatize(word)[0]
				except:
					word_lemma = word
				tokens[i] = word_lemma
				final = "-".join(tokens)
		else:
			final = lemma
	except:
		final = ''
	return final

In [212]:
processed_profession

['заключенная карлага',
 'мать азизы тубековны, теща турара рыскулова',
 'мулла',
 'колхозник',
 'единоличник',
 'спиртзавод, возчик',
 'школа, учитель',
 'колхоз, счетовод',
 'крестьянин-единоличник',
 'сэс, инженер',
 '35 ст, рядовой',
 'крестьянин',
 '1 кп, 1 кд',
 'тюрьма, ст бригадир столярного цеха',
 'умз, плотник',
 'гублеском, агент',
 'яманелгинский леспромхоз, конюх',
 'солдат',
 'уфимский район, налоговый инспектор',
 'псаломщик',
 '5-й лесозавод, молотобоец',
 'священнослужитель',
 'лесозавод, плотник',
 'слесарь',
 'плодоовощной завод, механик',
 'колхоз красный украинец , столяр',
 'дьякон',
 'колхоз смычка , кузнец',
 'совхоз красная башкирия , рабочий',
 'ветфельдшер',
 'трест башзолото , старатель',
 'кустарь-кожевенник',
 'мельзавод №3',
 'священник',
 'стройтрест №17, рабочий',
 'колхоз знамя труда , колхозник',
 'кирпичный завод, рабочий',
 'артель «новая жизнь», рабочий',
 '710-й сп, красноармеец',
 'рабочий',
 'благоварский район, ст агроном',
 'кз им ворошилова,

In [15]:
def right_lemmatisation(token):
	if "-" in token:
		new_word = hyphen_lemmatizer(token)
	else:
		try:
			new_word = m.lemmatize(token)[0]
		except:
			new_word = ''
	return new_word

В нее приходит слово, потом получается лемма. Чтобы не анализироать каждое слово, мы какие-то разборы соханяем и потом их достаем из кэша, чтобы каждый раз их не запускать. Берем словарь `m_cache`, проверяется, есть ли там слово. Если слова там нет, то мы его лемматизируем, делаем ему разбор (то, что происходит под `if`). В это месте мы лемматизируем слова, которых в словарике нет. Записываем их в этот кэш. Если слова в кеше, то `if` не срабатывает и мы возвращаем слово обратно. Это - способ не разбирать слово много раз, если данных много. Ускорение работы программы. Чтобы не применять `mystem` ко всем текстам. 
Кроме того, в функции проверяется одушеленность и и выдаче эта информация фигурирует как 4 элемент списка ( `[3] - anim`). Описывается как `TRUE\ FALSE`.

In [16]:
def cashing(w):
	global m_cache
	if w not in m_cache:
		lemma = right_lemmatisation(w)
		pos, list_of_cases, anim = '', '', False
		try:
			pos = m.analyze(w)[0]['analysis'][0]['gr'].split(",")[0]
			list_of_cases = m.analyze(w)[0]['analysis'][0]['gr'].split("=")[1]
			animacy = m.analyze(w)[0]['analysis'][0]['gr'].split(",")[2]
			anim = True if "неод" not in animacy and "од" in animacy and w not in false_subs else False
		except:
			pass
		m_cache[w] = (
			lemma if lemma else '',
			pos,
			list_of_cases,
			anim
		)
	return m_cache[(w)]

В этой функции список разбивается на элементы по `","` или `";"` (было `["яманелгинский леспромхоз", "конюх"]`, стало `["яманелгинский леспромхоз"], ["конюх"]`), если такие знаки есть:
- Если `animacy_based_resolution` выдала `'zero'`, то в этих элементах списка не содержится одушевленных сущ. Так бывает редко, поэтому функция начинает искать там название профессии по минимальной длине слова. 
- Если по результатам `animacy_based_resolution` получилось больше, чем 1, то функция берет снова эти элементы (сколько их там получилось, 2 и больше) и тоже выбирает из них наименьший. 
- Если в выдаче `animacy_based_resolution` значение 1 (скорее всего, это самый популярный вариант), то это полученное знаечение и присваивается в результате работы `else`.

После этого- в функцию `case_based_resolution`

Если таких знаков нет:

Обращается сразу к функции `case_based_resolution`.

На финальном этапе производится приведение профессий ж.р. в м.р. (`women_postprocessing`).

In [17]:
def final_profession_resolution(profession):
	final_dict = {}
	if "," in profession or ":" in profession:
		if "," in profession:
			professions = profession.split(",")
		elif ":" in profession:
			professions = profession.split(":")
		if animacy_based_resolution(professions) == "zero":
			final_profession = min(professions, key = len)
		elif type(animacy_based_resolution(professions)) == list:
			prs = animacy_based_resolution(professions)
			final_profession = min(prs, key = len)
		else:
			final_profession = animacy_based_resolution(professions)
		final_profession = case_based_resolution(final_profession)
	else:
		final_profession = case_based_resolution(profession)
	final_dict["prof_before_women_postproc"] = final_profession.strip()
	final_dict["prof_after_women_postproc"] = women_postprocessing(final_profession.strip())
	return final_dict

Разрешение на основании одушевленности. Создается словарь и в него складываются части `professions`, в которых есть одушевленные слова (которые были определены в функции `cashing`). Если неодушевленные, то в словарь не попадают.  
Например, если у нас есть `"яманелгинский леспромхоз, конюх"`, то в функцию подастся `["яманелгинский леспромхоз"], ["конюх"]`. В этом случае в первом списке нет одушевленных, а во втором есть, значит, результат `sum(dicti.values()` будет 1+0=1. В `[0]` распаковывается список и достается то, что =1.
Если одуш. нет, то `'zero'`

In [18]:
def animacy_based_resolution(professions):
	dicti = {}
	for potential_profession in professions:
		tokens = potential_profession.split(" ")
		animacies = [cashing(token)[3] for token in tokens]
		if True in animacies:
			dicti[potential_profession] = 1
		else:
			dicti[potential_profession] = 0
	if sum(dicti.values()) == 1:
		return [profession for profession in dicti if dicti[profession] == 1][0]
	elif sum(dicti.values()) > 1:
		return [profession for profession in dicti if dicti[profession] == 1]
	else:
		return "zero"

Эта функция используется на заключительном этапе `final_profession_resolution` в любом из сценариев развития. В названии профессии, разбитом по пробелу, функция выбирает слова, которые:
- являются существительными и в "им,ед" и не входят ни в "положительные", ни в "отрицательные" исключения.
- являются существительными и в "твор" и не входят ни в "положительные", ни в "отрицательные" исключения.

Иначе просто возвращает изанчальную форму профессии. 

In [19]:
def case_based_resolution(profession):
	for word in profession.split(" "):
		if (cashing(word)[1] == "S" and "им,ед" in cashing(word)[2] and word not in false_subs and cashing(word)[3] == True) or word in true_subs:
			return word
		elif (cashing(word)[1] == "S" and "твор" in cashing(word)[2] and word not in false_subs and cashing(word)[0] not in false_subs) or word in true_subs:
			return cashing(word)[0]
	return profession

Функция `women_postprocessing` изменяет окончания женского рода на окончания мужского.

In [20]:
def women_postprocessing(profession):
	hushing = ["ш", "щ", "ч"]
	if profession[-3:] == "ица":
		new_profession = profession[:-2] + "к"
	elif profession[-5:] == "истка":
		new_profession = profession[:-2]
	elif profession[-2:] == "ая":
		if profession[-3] not in hushing:
			new_profession = profession[:-2] + "ый"
		else:
			new_profession = profession[:-2] + "ий"
	elif profession[-4:] == "аяся":
		if profession[-5] not in hushing:
			new_profession = profession[:-4] + "ыйся"
		else:
			new_profession = profession[:-2] + "ий"
	elif profession[-3:] == "тка":
		new_profession = profession[:-2]
	elif profession[-3:] == "иха":
		new_profession = profession[:-3]
	elif profession[-3:] == "иня":
		new_profession = profession[:-3]
	elif profession[-2:] == "ка":
		new_profession = profession[:-2]
	else:
		new_profession = profession
	return new_profession

## 1.2 Тест кода

In [4]:
def professions_detection(line):
	profession = get_batch_scores(line)
	preprocessed_profession = basic_preprocessing(profession)
	final_profession = final_profession_resolution(preprocessed_profession)
	return final_profession

In [ ]:
final_profession = []
for occupation in tqdm(occupations):
    final_profession.append(professions_detection(occupation))

In [ ]:
all_profession = pd.DataFrame(final_profession)
all_profession["init_prof"] = occupations

In [ ]:
all_profession.to_csv('occupations_processed.csv')

In [51]:
all_profession

,prof_before_women_postproc,prof_after_women_postproc,init_prof
0,заключенная,заключенный,заключенная Карлага
1,мать,мать,"(мать Азизы Тубековны, теща Турара Рыскулова)"
2,мулла,мулла,мулла
3,колхозник,колхозник,колхозник
4,единоличник,единоличник,единоличник
...,...,...,...
667019,бюро,бюро,Нач. бюро технического нормирования локомотивн...
667020,проповедник,проповедник,проповедник секты баптистов
667021,футболист,футболист,футболист
667022,медик,медик,"медик, зам. нач. управления высшими учебными з..."


In [7]:
occupations_processed = pd.read_csv('/Users/annalevina/Desktop/occupations_processed.csv', low_memory=False) 

In [6]:
occupations_processed["prof_after_women_postproc"].value_counts()/occupations_processed["prof_after_women_postproc"].shape[0]

рабочий                                          0.048328
колхозник                                        0.030077
бухгалтер                                        0.027554
красноармеец                                     0.026521
начальник                                        0.022807
                                                   ...   
инж -механик актюбин аэропорта                   0.000001
вновь арестован 23 02 1944 г                     0.000001
нач ключевского земотряда кразу                  0.000001
рай касса                                        0.000001
работал в с дербентское усть-абаканского р-на    0.000001
Name: prof_after_women_postproc, Length: 35703, dtype: float64

In [9]:
top400 = occupations_processed['prof_after_women_postproc'].value_counts().head(400)

In [17]:
top400

рабочий               32236
колхозник             20062
бухгалтер             18379
красноармеец          17690
начальник             15213
                      ...  
землемер                135
бюро                    135
десятник-строитель      134
арендатор               134
управдом                134
Name: prof_after_women_postproc, Length: 400, dtype: int64